In [20]:
pos_tag_map = {
    "ABL": "Ablative",
    "ABN": "Adverb",
    "ABX": "Adverb (unspecified)",
    "AP": "Adjective",
    "APg": "Adjective (generic)",
    "AT": "Article",
    "BE": "Verb (to be)",
    "BED": "Past participle of 'to be'",
    "BEDZ": "Third person singular past tense of 'to be'",
    "BEG": "Gerund of 'to be'",
    "BEM": "Present participle of 'to be'",
    "BEN": "Noun (gerund)",
    "BER": "Present participle of 'to be'",
    "BEZ": "Third person singular present tense of 'to be'",
    "CC": "Coordinating conjunction",
    "CD": "Cardinal number",
    "CDg": "Cardinal number (generic)",
    "CS": "Subordinating conjunction",
    "DO": "Auxiliary verb 'do'",
    "DOD": "Past participle of 'to do'",
    "DOZ": "Third person singular present tense of 'to do'",
    "DT": "Determiner",
    "DTI": "Determiner (indefinite)",
    "DTS": "Determiner (specific)",
    "DTX": "Determiner (unknown)",
    "DTg": "Determiner (generic)",
    "EX": "Existential 'there'",
    "HV": "Auxiliary verb 'have'",
    "HVD": "Past participle of 'to have'",
    "HVG": "Gerund of 'to have'",
    "HVN": "Noun (gerund)",
    "HVZ": "Third person singular present tense of 'to have'",
    "IN": "Preposition or subordinating conjunction",
    "JJ": "Adjective",
    "JJR": "Comparative adjective",
    "JJS": "Superlative adjective",
    "JJT": "Adjective (temporal)",
    "JJg": "Adjective (generic)",
    "MD": "Modal verb",
    "NEG": "Negation",
    "NIL": "Null",
    "NN": "Noun (singular)",
    "NNS": "Noun (plural)",
    "NNSg": "Noun (plural generic)",
    "NNg": "Noun (singular generic)",
    "NP": "Proper noun (singular)",
    "NPS": "Proper noun (plural)",
    "NPSg": "Proper noun (plural generic)",
    "NPg": "Proper noun (singular generic)",
    "NR": "Noun (reduced)",
    "NRS": "Noun (reduced plural)",
    "NRg": "Noun (reduced generic)",
    "OD": "Ordinal number",
    "PN": "Pronoun",
    "PNg": "Pronoun (generic)",
    "PPL": "Participle",
    "PPLS": "Participle (plural)",
    "PPO": "Personal pronoun (objective)",
    "PPS": "Personal pronoun (subjective)",
    "PPSS": "Personal pronoun (subjective singular)",
    "PPg": "Personal pronoun (generic)",
    "PPgg": "Personal pronoun (generic plural)",
    "QL": "Quantifier",
    "QLP": "Quantifier (postdeterminer)",
    "RB": "Adverb",
    "RBR": "Comparative adverb",
    "RBT": "Temporal adverb",
    "RBg": "Adverb (generic)",
    "RN": "Adverb (reduced)",
    "RP": "Particle",
    "TO": "Infinitive marker",
    "UH": "Interjection",
    "VB": "Verb (base form)",
    "VBD": "Past tense verb",
    "VBG": "Gerund",
    "VBN": "Past participle",
    "VBZ": "Third person singular present tense verb",
    "WDT": "Wh-determiner",
    "WPO": "Wh-pronoun (objective)",
    "WPS": "Wh-pronoun (subjective)",
    "WPg": "Wh-pronoun (generic)",
    "WQL": "Wh-quantifier",
    "WRB": "Wh-adverb",
    "pct": "Percentage"
}
tag_dict = {
    key : 0 for key in pos_tag_map.keys()
}
pos_tag_we_keys = list(pos_tag_map.keys());pos_tag_we_keys.append('<E>')
tag_dict_we = {
    key : 0 for key in pos_tag_we_keys
}
itos = {i+1:v for i,v in enumerate(pos_tag_map.keys())}
itos[0] = '<S>'
itos[len(tag_dict)+1] = '<E>'
stoi = {v:k for k,v in itos.items()}

# stoi

In [21]:
import xml.etree.ElementTree as ET
import numpy as np
import random
import copy
import math

sample_file = ET.parse('brown_tei/Corpus.xml')
root = sample_file.getroot()
tei = root.findall('{http://www.tei-c.org/ns/1.0}TEI')
total_words = 0
states = set({})
sentences = []
words_map = {'<S>':0,'<E>':1}
for t in tei:
    text = t.find('{http://www.tei-c.org/ns/1.0}text')
    body = text.find('{http://www.tei-c.org/ns/1.0}body')
    for paras in body:
        for sentence in paras:
            if sentence.find('{http://www.tei-c.org/ns/1.0}mw') != None:
                continue
            tags_in_sentence = ['<S>']
            words_in_sentence = ['<S>']
            for word in sentence:
                total_words += 1
                words_map[word.text] = len(words_map)
                states.add(word.attrib["type"])
                tags_in_sentence.append(word.attrib["type"])
                words_in_sentence.append(word.text)
                # words[word][word.attrib["type"]] += 1
                # print(f'{word.text} --> {pos_tag_map[word.attrib["type"]]}')
            tags_in_sentence.append('<E>')
            words_in_sentence.append('<E>')
            sentences.append({"tags":tags_in_sentence,"words":words_in_sentence})
random.shuffle(sentences)
tr = math.ceil(0.8*len(sentences))
tr_sentences = sentences[:tr]
tst_sentences = sentences[tr:]
unique_words = len(words_map) + 2

print("---- data processed summary ----")
print(f'total samples : {len(tei)}')
print(f'total words   : {total_words}')
print(f'total distinc words   : {len(words_map)}')
print(f'total tags    : {len(states)}')
print(f'total valid sentences : {len(sentences)}')
print(f'training sentence set : {len(tr_sentences)}')
print(f'testing sentence set  : {len(tst_sentences)}')

---- data processed summary ----
total samples : 500
total words   : 1081150
total distinc words   : 54496
total tags    : 84
total valid sentences : 52686
training sentence set : 42149
testing sentence set  : 10537


dimensionss of emission count : 49165 84.0
dimensionss of transition count : 85 (84 tags + <S>) 85.0 (84 tags + <E>)
dimensionss of emission count (eff) : 54498 84
dimensionss of transition count (eff) : 85 85


In [22]:
# Find the transition and emission matrix

import matplotlib.pyplot as plt
%matplotlib inline

emission_count = {'<S>':copy.deepcopy(tag_dict),'<E>':copy.deepcopy(tag_dict)}
transition_count = {'<S>':copy.deepcopy(tag_dict_we)}
emission_arr = np.zeros((unique_words,len(states)))
transition_arr = np.zeros((len(states) +1,len(states) + 1))
for key in pos_tag_map.keys():
    transition_count[key] = copy.deepcopy(tag_dict_we)
for sentence in tr_sentences:
    context = '<S>'
    for i in range(1,len(sentence["words"])):
        w = sentence["words"][i]
        t = sentence["tags"][i]
        if t not in stoi:
            print(f'debug tag -> {t}')
        transition_count[context][t] += 1
        # print(f'{context} {t} {transition_count[context][t]}')
        context = t
        if w=='<E>':
            continue
        if w not in emission_count:
            emission_count[w] = copy.deepcopy(tag_dict)
        emission_count[w][t] += 1
for w in emission_count:
    for c in emission_count[w]:
        i = words_map[w]
        emission_arr[i][stoi[c]-1] = emission_count[w][c]
for i,s in enumerate(transition_count):
    for c in transition_count[s]:
        transition_arr[i][stoi[c]-1] = transition_count[s][c]


total_unknown_words = 0
for i,rows in enumerate(emission_arr):
    if emission_arr[i].sum()<1:
        # print(f'debug emission_sum -> {emission_arr[i].sum()} i -> {i}')
        total_unknown_words+=1
        continue
    emission_arr[i] = emission_arr[i]/ emission_arr[i].sum()
# print(f'total_unknown_words {total_unknown_words}')
for i,rows in enumerate(transition_arr):
    transition_arr[i] = transition_arr[i]/ transition_arr[i].sum()

#Finding the emissions transition and start probabilities

real_emission = copy.deepcopy(tag_dict)
for w in emission_count:
    for t in emission_count[w]:
        real_emission[t] += emission_count[w][t]
for w in emission_count:
    for t in emission_count[w]:
        emission_count[w][t] = emission_count[w][t]/real_emission[t]

start_prob = {}
for s in transition_count['<S>']:
    start_prob[s] = transition_count['<S>'][s]/sum(transition_count['<S>'].values())
for prev in transition_count:
    # if prev == '<S>':
    #     continue
    for s in transition_count[prev]:
        transition_count[prev][s] = transition_count[prev][s]/sum(transition_count[prev].values())
# coded up an inefficent way of finding the dimmensions

avg_emm_val = 0
for k,v in emission_count.items():
    avg_emm_val += len(v)
avg_emm_val /= len(emission_count)
avg_trans_val = 0
for k,v in transition_count.items():
    avg_trans_val += len(v)
avg_trans_val /= len(transition_count)
print(f'dimensionss of emission count : {len(emission_count)} {avg_emm_val}')
print(f'dimensionss of transition count : {len(transition_count)} (84 tags + <S>) {avg_trans_val} (84 tags + <E>)')

# effiencent way
print(f'dimensionss of emission count (eff) : {len(emission_arr)} {len(emission_arr[0])}')
print(f'dimensionss of transition count (eff) : {len(transition_arr)} {len(transition_arr[0])}')

emission_arr[49086].sum()

# VISUALIZATION

trans_cnt_arr = [[transition_count[tag][v] for v in transition_count[tag]] for tag in transition_count]
plt.figure(figsize=(8,8)) # :) if you want to look at larger image maybe put (16,16) or (32,32)
plt.imshow(trans_cnt_arr,cmap='Oranges')
arr_tags = ['S'];arr_tags.extend([key for key in pos_tag_map.keys()]);arr_tags.append('<E>')

# Not sure what i wanted to do :P (it's modified version of visuzation code) ⚠️

# for r in trans_cnt_arr:
#     print(r)
# plt.figure(figsize=(16,16))
# plt.imshow(trans_cnt_arr,cmap='Oranges')
# arr_tags = ['S'];arr_tags.extend([key for key in pos_tag_map.keys()]);arr_tags.append('<E>')
# for i in arr_tags[:-1]:
#     for j in arr_tags[1:]:
#         chstr = i+j
#         plt.text(j,i,chstr,ha="center",va="bottom",color="gray")
#         plt.text(j,i,transition_count[i][j],ha="center",va="top",color="gray")
# plt.show()

In [23]:
# (Failure) i converted the hashmap to array, thinking it will be easy to manupulate array  ⚠️
# I am facing indexing issue (i did verify it above with the emission count len)  ⚠️

# def viterbi(sentence,emission_arr,transition_arr,states):
#     pstates = ['<S>']
#     for i in range(1,len(sentence["words"])):
#         w = sentence["words"][i]
#         temp_p = []
#         for state in states:
#             if w=='<S>':
#               trans_p = 1
#             else:
#               trans_p = transition_arr[stoi[pstates[-1]]-1][stoi[state]-1]
#             i = words_map[w]
#             emit_p = emission_arr[i][stoi[state]-1]
#             state_p = trans_p * emit_p
#             temp_p.append(state_p)
#         pmax = max(temp_p)
#         state_max = states[temp_p.index(pmax)]
#         pstates.append(state_max)
#     return pstates

In [24]:
from operator import indexOf
def viterbi(sentence,emission_count,transition_count,states):
    pstates = ['<S>']
    for i in range(1,len(sentence["words"])):
        w = sentence["words"][i]
        temp_p = []
        for state in states:
          trans_p = transition_count[pstates[-1]][state]
          i = words_map[w]
          emit_p = emission_count[w][state] if w in emission_count else 1 # on the go data smoothing :)
          state_p = trans_p * emit_p
          temp_p.append((state_p,state))
        pmax = max(temp_p,key = lambda x:x[0])
        state_max = pmax[1]
        pstates.append(state_max)
    return pstates

# FOR CHECKING AND TESTING

ans = viterbi(tst_sentences[0],emission_count,transition_count,list(states))
print(list(zip(ans,tst_sentences[0]["words"])))
print(list(zip(tst_sentences[0]["tags"],tst_sentences[0]["words"])))

[('<S>', '<S>'), ('pct', 'Unwholesome'), ('NN', 'family'), ('NNS', 'relations'), ('pct', ','), ('VBN', 'broken'), ('NNS', 'homes'), ('pct', ','), ('CC', 'and'), ('JJ', 'undesirable'), ('NN', 'community'), ('VBZ', 'influences'), ('MD', 'may'), ('RB', 'also'), ('BE', 'be'), ('VBG', 'contributing'), ('NNS', 'factors'), ('pct', '.'), ('BEN', '<E>')]
[('<S>', '<S>'), ('JJ', 'Unwholesome'), ('NN', 'family'), ('NNS', 'relations'), ('pct', ','), ('VBN', 'broken'), ('NNS', 'homes'), ('pct', ','), ('CC', 'and'), ('JJ', 'undesirable'), ('NN', 'community'), ('NNS', 'influences'), ('MD', 'may'), ('RB', 'also'), ('BE', 'be'), ('VBG', 'contributing'), ('NNS', 'factors'), ('pct', '.'), ('<E>', '<E>')]


In [26]:
import math
def viterbi_real(sentence,emission_count,transition_count,states):
  cache = {}
  def hlp(i,cur_state):
    if i>=len(sentence["words"]):
      return (0,[])
    if (i,cur_state) in cache:
      return cache[(i,cur_state)]
    max_prob = -float('inf')
    best_path = None
    for ns in states:
      (prob, path) =  hlp(i+1,ns)
      tp = math.log(transition_count[cur_state][ns]) if transition_count[cur_state][ns]!=0 else math.log(0.000000000000001)
      if sentence["words"][i] in emission_count:
        ep = math.log(emission_count[sentence["words"][i]][ns]) if emission_count[sentence["words"][i]][ns]!=0 else math.log(0.000000000000001)
      if sentence["words"][i] not in emission_count:
        ep = math.log(0.000000000000001)
      state_p =  tp + ep + prob
      if state_p > max_prob:
        max_prob = state_p
        best_path = [cur_state] + copy.deepcopy(path)
    if i==1:
      max_prob += math.log(start_prob[cur_state]) if start_prob[cur_state]!=0 else math.log(0.00000001)
    cache[(i,cur_state)] = (max_prob,best_path)
    return cache[(i,cur_state)]
  max_prob,best_path = -float("inf"),[]
  for i in states:
    prob,bp = hlp(1,i)
    if prob > max_prob:
      max_prob = prob
      best_path = bp
  return max_prob,best_path

# FOR CHECKING AND TESTING
print(emission_count["When"])
(i,ans) = viterbi_real(tst_sentences[0],emission_count,transition_count,states)
ans = ['<S>'] + ans[1:]
print(i,ans)
print(list(zip(ans,tst_sentences[0]["words"])))
print(list(zip(tst_sentences[0]["tags"],tst_sentences[0]["words"])))

{'ABL': 0.0, 'ABN': 0.0, 'ABX': 0.0, 'AP': 0.0, 'APg': 0.0, 'AT': 0.0, 'BE': 0.0, 'BED': 0.0, 'BEDZ': 0.0, 'BEG': 0.0, 'BEM': 0.0, 'BEN': 0.0, 'BER': 0.0, 'BEZ': 0.0, 'CC': 0.0, 'CD': 0.0, 'CDg': 0.0, 'CS': 0.0, 'DO': 0.0, 'DOD': 0.0, 'DOZ': 0.0, 'DT': 0.0, 'DTI': 0.0, 'DTS': 0.0, 'DTX': 0.0, 'DTg': 0.0, 'EX': 0.0, 'HV': 0.0, 'HVD': 0.0, 'HVG': 0.0, 'HVN': 0.0, 'HVZ': 0.0, 'IN': 0.0, 'JJ': 0.0, 'JJR': 0.0, 'JJS': 0.0, 'JJT': 0.0, 'JJg': 0.0, 'MD': 0.0, 'NEG': 0.0, 'NIL': 0.0, 'NN': 0.0, 'NNS': 0.0, 'NNSg': 0.0, 'NNg': 0.0, 'NP': 0.0, 'NPS': 0.0, 'NPSg': 0.0, 'NPg': 0.0, 'NR': 0.0, 'NRS': 0.0, 'NRg': 0.0, 'OD': 0.0, 'PN': 0.0, 'PNg': 0.0, 'PPL': 0.0, 'PPLS': 0.0, 'PPO': 0.0, 'PPS': 0.0, 'PPSS': 0.0, 'PPg': 0.0, 'PPgg': 0.0, 'QL': 0.0, 'QLP': 0.0, 'RB': 0.0, 'RBR': 0.0, 'RBT': 0.0, 'RBg': 0.0, 'RN': 0.0, 'RP': 0.0, 'TO': 0.0, 'UH': 0.0, 'VB': 0.0, 'VBD': 0.0, 'VBG': 0.0, 'VBN': 0.0, 'VBZ': 0.0, 'WDT': 0.0, 'WPO': 0.0, 'WPS': 0.0, 'WPg': 0.0, 'WQL': 0.0, 'WRB': 0.13705426356589148, 'pct':

In [ ]:
def accuracy(tst_sentences,emission_count,transition_count,states):
    corr_emi = 0
    correct_sentences = 0
    correct_r_sentences = 0
    correct = 0
    r_correct = 0
    total = 0
    new_words = 0
    for j,sentence in enumerate(tst_sentences):
        print(f'\rprocessed {j/len(tst_sentences)*100}%')
        ans = viterbi(sentence,emission_count,transition_count,list(states))
        (_,r_ans) = viterbi_real(sentence,emission_count,transition_count,list(states))
        r_ans = ['<S>'] + r_ans[1:] + ['<E>']
        for i in range(1,len(sentence["words"])-1):
            if ans[i] == sentence["tags"][i]:
                correct += 1
            if r_ans[i] == sentence["tags"][i]:
                r_correct += 1
            key_value = list(emission_count[sentence["words"][i]].items()) if sentence["words"][i] in emission_count else -1
            if key_value == -1:
                new_words += 1
            if key_value != -1:
              pred_key_index = key_value.index(max(key_value,key = lambda x:x[1]))
              (state,count) = key_value[pred_key_index]
              # print(state,count)
              if state==sentence["tags"][i]:
                  corr_emi += 1
            total += 1
        if "".join(ans[1:-1]) == "".join(sentence["tags"][1:-1]):
            correct_sentences += 1
        if "".join(r_ans[1:-1]) == "".join(sentence["tags"][1:-1]):
            correct_r_sentences += 1
    t_acc = correct/total*100
    t_r_acc = r_correct/total*100
    s_acc = correct_sentences/len(tst_sentences)*100
    s_r_acc = correct_r_sentences/len(tst_sentences)*100
    te_acc = corr_emi/total*100
    return (t_acc,s_acc,te_acc,t_r_acc,new_words,total,s_r_acc)

(t_acc,s_acc,te_acc,t_r_acc,new_words,total,s_r_acc) = accuracy(tst_sentences,emission_count,transition_count,states)
print(f'token level acc (greedy algo): {t_acc}% \nsentence level acc (greedy algo):  {s_acc}%\ntoken level acc (based only on emission):{te_acc}\ntoken level accuracy (viterbi algorithm) :{t_r_acc}\nnew words:{new_words}\ntotal words:{total}\\
      \nsentence level accuracy (viterbi algorithm): {s_r_acc}')

Streaming output truncated to the last 5000 lines.
processed 47.186106102306155%
processed 47.19559646958337%
processed 47.20508683686059%
processed 47.2145772041378%
processed 47.22406757141501%
processed 47.23355793869223%
processed 47.243048305969445%
processed 47.252538673246654%
processed 47.26202904052387%
processed 47.27151940780108%
processed 47.281009775078296%
processed 47.290500142355505%
processed 47.29999050963272%
processed 47.30948087690994%
processed 47.31897124418715%
processed 47.32846161146436%
processed 47.33795197874158%
processed 47.347442346018795%
processed 47.356932713296004%
processed 47.36642308057321%
processed 47.37591344785043%
processed 47.385403815127646%
processed 47.394894182404855%
processed 47.40438454968207%
processed 47.41387491695929%
processed 47.4233652842365%
processed 47.43285565151371%
processed 47.44234601879093%
processed 47.451836386068145%
processed 47.461326753345354%
processed 47.47081712062256%
processed 47.48030748789978%
processed 47